In [1]:
#lectura de datos #SOLUCION PRIMER PUNTO
import pandas as pd
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

from sklearn.metrics import confusion_matrix
import json

# Ruta al archivo zip 
train_data_zip = '/Users/juan1/Documents/GitHub/20204-2-LAB-02-prediccion-del-default-usando-logreg-jdlodonog/files/input/train_data.csv.zip'
test_data_zip='/Users/juan1/Documents/GitHub/20204-2-LAB-02-prediccion-del-default-usando-logreg-jdlodonog/files/input/test_data.csv.zip'
# Nombre del archivo CSV dentro del archivo zip
#csv_file_name = 'files/input/test_data.csv'

In [2]:
train_data=pd.read_csv(
        train_data_zip,
        index_col=False,
        compression='zip')

test_data=pd.read_csv(
        test_data_zip,
        index_col=False,
        compression='zip')

In [3]:
#renombrar columna de dataframe default payment next month" a "default"
df_train=train_data.copy()
df_test=test_data.copy()

In [4]:
df_train=df_train.rename(columns={'default payment next month':'default'})
df_train.head()
df_test=df_test.rename(columns={'default payment next month':'default'})
df_test

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0
2,11,200000,2,3,2,34,0,0,2,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,15,250000,1,1,2,29,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,16,50000,2,3,3,23,1,2,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,29984,20000,1,2,1,44,-2,-2,-2,-2,...,2882,9235,1719,2890,2720,2890,9263,1824,1701,0
8996,29987,360000,1,1,2,35,-1,-1,-2,-2,...,0,0,0,0,0,0,0,0,0,0
8997,29990,150000,1,1,2,35,-1,-1,-1,-1,...,780,0,0,9054,0,783,0,0,0,0
8998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1


In [5]:
#remover la columna de los data set
df_train=df_train.drop(columns=['ID'])
df_test=df_test.drop(columns=['ID'])
df_train.head()
df_test.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,20000,1,3,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,0
2,200000,2,3,2,34,0,0,2,0,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,250000,1,1,2,29,0,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,50000,2,3,3,23,1,2,0,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0


In [6]:
print(df_train.shape)
print(df_test.shape)

(21000, 24)
(9000, 24)


In [7]:
#Elimine los registros con informacion no disponible.
dfc_train= df_train.dropna()
dfc_test= df_test.dropna()
dfc_train.head()
dfc_test.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,20000,1,3,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,0
2,200000,2,3,2,34,0,0,2,0,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,250000,1,1,2,29,0,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,50000,2,3,3,23,1,2,0,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0


In [8]:
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others"

dfc_train['EDUCATION']=dfc_train['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
dfc_test['EDUCATION']=dfc_test['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

#se seleccionan 
dfc_train=dfc_train.loc[dfc_train['EDUCATION']!=0]
dfc_test=dfc_test.loc[dfc_test['EDUCATION']!=0]

dfc_train=dfc_train.loc[dfc_train['MARRIAGE']!=0]
dfc_test=dfc_test.loc[dfc_test['MARRIAGE']!=0]

In [9]:
print(dfc_train.shape)
print(dfc_test.shape)

(20953, 24)
(8979, 24)


In [10]:
X_train = dfc_train.drop(columns=["default"]) 
y_train = dfc_train["default"]               

X_test = dfc_test.drop(columns=["default"])  
y_test =dfc_test["default"]

In [11]:
X_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,310000,1,3,1,32,0,0,0,0,0,...,110375,84373,57779,14163,8295,6000,4000,3000,1000,2000
1,10000,2,3,1,49,-1,-1,-2,-1,2,...,-748,1690,1138,930,0,0,2828,0,182,0
2,50000,1,2,1,28,-1,-1,-1,0,-1,...,46257,45975,1300,43987,0,46257,2200,1300,43987,1386
3,80000,2,3,1,52,2,2,3,3,3,...,40101,40748,39816,40607,3700,1600,1600,0,1600,1600
4,270000,1,1,2,34,1,2,0,0,2,...,20924,22448,15490,17343,0,4000,2000,0,2000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2,1,27,2,-1,-1,-1,0,...,776,1580,804,728,752,800,1580,0,700,700
20996,130000,1,2,2,41,0,0,0,0,0,...,123746,123107,42897,39378,4442,5200,5012,2500,5000,2000
20997,50000,1,3,2,23,0,0,0,0,0,...,36023,28967,29829,30046,1973,1426,1001,1432,1062,997
20998,90000,2,3,2,25,0,0,0,0,0,...,5613,5613,10113,10113,3000,3000,0,4500,0,3440


In [12]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Ajusta un modelo de bosques aleatorios (random forest).
from sklearn.pipeline import Pipeline

numerical_features= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]

#Se define el preprocesaror para aplicar el one-hot encoding a las variables categoricas
preprocessor = ColumnTransformer( transformers=[ ('encoder', OneHotEncoder(), ['SEX', 'EDUCATION','MARRIAGE']), 
                                                ('scaler', MinMaxScaler() ,numerical_features)
    ],
    remainder = "passthrough")

'''
# Crea un objeto pipeline
pipe = make_pipeline(preprocessor,
                    SelectKBest(score_func=f_classif),
                    LogisticRegression(max_iter=1000, random_state=42)
                    )           
'''

 #Creación del Pipeline
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("selectkbest", SelectKBest(score_func=f_classif)),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

pipe.fit(X_train,y_train)
print("Modelo entrenado. Precisión en test:", pipe.score(X_test, y_test))

Modelo entrenado. Precisión en test: 0.820469985521773


In [13]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo
#from scipy.stats import randint
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
import warnings

warnings.filterwarnings("ignore")


# define el espacio de parametros para optimizar
#NOTA: Esto es una forma
param_grid = {
#'feature_selection__k': [5, 10, 15], # Número de características a seleccionar 
#'classifier__C': [0.1, 1, 10], # Parámetro de regularización de la regresión logística   
#'classifier__solver': ['liblinear', 'lbfgs'],
"selectkbest__k": [1],
"classifier__C": [0.6],
#'LogisticRegression__random_state':[42500]  
}

# Crear el objeto GridSearchCV con validación cruzada de 10 divisiones y precisión balanceada 
grid_search = GridSearchCV(pipe, param_grid, cv=10,scoring='balanced_accuracy', n_jobs=-1 )#faltaba numero de jons

# Entrenar el modelo con GridSearchCV 
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor modelo 
best_params = grid_search.best_params_ #coge los mejores parametros del estimador
model = grid_search.best_estimator_  #en este caso solo se tiene un estimador 
best_score = grid_search.best_score_
model=grid_search

In [14]:
best_params

{'classifier__C': 0.6, 'selectkbest__k': 1}

In [15]:
# Paso 5.
# Guarde el modelo como "files/models/model.pkl".

import os 
import pickle
# Supongamos que best_model es tu modelo entrenado # Crear una carpeta si no existe 
output_dir = '../files/models' 

if not os.path.exists('../files/models'):
        os.makedirs('../files/models')

import gzip
import pickle
compressed_model_path = '../files/models/model.pkl.gz'

with gzip.open(compressed_model_path, 'wb') as f:
        pickle.dump(model, f)

In [16]:
y_train_pred = model.predict(X_train) 
y_test_pred = model.predict(X_test)

# Calcular métricas para el conjunto de entrenamiento
metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': float(precision_score(y_train, y_train_pred, zero_division=0)),
        'balanced_accuracy': float(balanced_accuracy_score(y_train, y_train_pred)),
        'recall': float(recall_score(y_train, y_train_pred, zero_division=0)),
        'f1_score': float(f1_score(y_train, y_train_pred, zero_division=0))
    }

metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': float(precision_score(y_test, y_test_pred, zero_division=0)),
        'balanced_accuracy': float(balanced_accuracy_score(y_test, y_test_pred)),
        'recall': float(recall_score(y_test, y_test_pred, zero_division=0)),
        'f1_score': float(f1_score(y_test, y_test_pred, zero_division=0))
    }

metrics = [metrics_train, metrics_test]

#guardar tipo json con pandas

    # Imprimir métricas en consola
print("Métricas del conjunto de entrenamiento:")
print(json.dumps(metrics_train, indent=4))
print("\nMétricas del conjunto de prueba:")
print(json.dumps(metrics_test, indent=4))

    # Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)


# Guardar las métricas en un archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(metrics_train) + '\n')
        f.write(json.dumps(metrics_test) + '\n')
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

Métricas del conjunto de entrenamiento:
{
    "type": "metrics",
    "dataset": "train",
    "precision": 0.6939338235294118,
    "balanced_accuracy": 0.6392682710528409,
    "recall": 0.31957671957671957,
    "f1_score": 0.43761773655991887
}

Métricas del conjunto de prueba:
{
    "type": "metrics",
    "dataset": "test",
    "precision": 0.7017913593256059,
    "balanced_accuracy": 0.6547057822566611,
    "recall": 0.34942287513116477,
    "f1_score": 0.46654991243432575
}


In [17]:
print(json.dumps(metrics_train, indent=4))
print(json.dumps(metrics_test, indent=4))

{
    "type": "metrics",
    "dataset": "train",
    "precision": 0.6939338235294118,
    "balanced_accuracy": 0.6392682710528409,
    "recall": 0.31957671957671957,
    "f1_score": 0.43761773655991887
}
{
    "type": "metrics",
    "dataset": "test",
    "precision": 0.7017913593256059,
    "balanced_accuracy": 0.6547057822566611,
    "recall": 0.34942287513116477,
    "f1_score": 0.46654991243432575
}


In [18]:
# PUNTO  7
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#
# Calcular las matrices de confusión para el conjunto de entrenamiento 
# Matriz de confusión para el conjunto de entrenamiento
cm_train = confusion_matrix(y_train, y_train_pred)

# Matriz de confusión para el conjunto de prueba
cm_test = confusion_matrix(y_test, y_test_pred)
# Crear la matriz de confusión formateada como diccionario
metrics = []

# Matriz de confusión para entrenamiento
metrics = [
    {
        'type': 'cm_matrix',
        'dataset': 'train',
        'true_0': {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
        'true_1': {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
    },
    {
        'type': 'cm_matrix',
        'dataset': 'test',
        'true_0': {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
        'true_1': {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
    }
    
]


# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(metrics_train) + '\n')
        f.write(json.dumps(metrics_test) + '\n')
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

#
print(f"Métricas guardadas en {output_path}")

Métricas guardadas en ../files/output/metrics.json
